In [8]:
import torch
import time

# 设置随机数种子，保证每次生成的随机数相同
torch.manual_seed(1234)

# 定义计算三角形面积的函数
def area(a, b, c):
    p = (a + b + c) / 2
    return torch.sqrt(p * (p - a) * (p - b) * (p - c))

# 定义计算三角形内切圆半径的函数
def inradius(a, b, c):
    return 2 * area(a, b, c) / (a + b + c)

# 定义计算三角形外接圆半径的函数
def circumradius(a, b, c):
    return a * b * c / (4 * area(a, b, c))

# 定义判断三角形是否符合要求的函数
def is_valid_triangle(a, b, c):
    if (a + b <= c) | (a + c <= b) | (b + c <= a):
        return False
    inrad = inradius(a, b, c)
    circumrad = circumradius(a, b, c)
    if inrad >= circumrad / 2:
        return False
    return True

# 定义蒙特卡洛方法计算符合要求的三角形概率的函数
def monte_carlo_gpu(N):
    # 将计算移动到 GPU 上
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    a = torch.empty(N, device=device).uniform_(0, 10)
    b = torch.empty(N, device=device).uniform_(0, 10)
    c = torch.empty(N, device=device).uniform_(0, 10)
    is_valid = (a + b > c) & (a + c > b) & (b + c > a)
    inrad = inradius(a, b, c)
    circumrad = circumradius(a, b, c)
    is_valid &= (inrad < circumrad / 2)
    count = torch.sum(is_valid)
    total = N
    start_time = time.time()
    prob = count / total
    end_time = time.time()
    elapsed_time = end_time - start_time
    return prob, elapsed_time

# 设置实验次数
N = 100000000

# 调用蒙特卡洛方法计算符合要求的三角形概率和计算时间
prob, elapsed_time = monte_carlo_gpu(N)

# 输出结果
print("符合要求的三角形概率为：", prob.item())
print("计算耗时：", elapsed_time, "秒")


符合要求的三角形概率为： 0.50003981590271
计算耗时： 2.7179718017578125e-05 秒


In [5]:
import random
import math
import time

# 设置随机数种子，保证每次生成的随机数相同
random.seed(1234)

# 定义计算三角形面积的函数
def area(a, b, c):
    p = (a + b + c) / 2
    return math.sqrt(p * (p - a) * (p - b) * (p - c))

# 定义计算三角形内切圆半径的函数
def inradius(a, b, c):
    return 2 * area(a, b, c) / (a + b + c)

# 定义计算三角形外接圆半径的函数
def circumradius(a, b, c):
    return a * b * c / (4 * area(a, b, c))

# 定义判断三角形是否符合要求的函数
def is_valid_triangle(a, b, c):
    if a + b <= c or a + c <= b or b + c <= a:
        return False
    inrad = inradius(a, b, c)
    circumrad = circumradius(a, b, c)
    if inrad >= circumrad / 2:
        return False
    return True

# 定义蒙特卡洛方法计算符合要求的三角形概率的函数
def monte_carlo(N):
    count = 0
    total = N
    start_time = time.time()
    for i in range(N):
        # 生成三个随机数作为三角形的边长
        a = random.uniform(0, 10)
        b = random.uniform(0, 10)
        c = random.uniform(0, 10)
        if is_valid_triangle(a, b, c):
            count += 1
    end_time = time.time()
    elapsed_time = end_time - start_time
    return count / total, elapsed_time

# 设置实验次数
N = 100000000

# 调用蒙特卡洛方法计算符合要求的三角形概率和计算时间
prob, elapsed_time = monte_carlo(N)

# 输出结果
print("符合要求的三角形概率为：", prob)
print("计算耗时：", elapsed_time, "秒")


符合要求的三角形概率为： 0.50002386
计算耗时： 159.02779030799866 秒


In [6]:
!nvidia-smi -q -d CLOCK


==============NVSMI LOG==============

Timestamp                                 : Fri Mar 10 13:16:26 2023
Driver Version                            : 530.30.02
CUDA Version                              : 12.1

Attached GPUs                             : 2
GPU 00000000:18:00.0
    Clocks
        Graphics                          : 1245 MHz
        SM                                : 1245 MHz
        Memory                            : 877 MHz
        Video                             : 1117 MHz
    Applications Clocks
        Graphics                          : 1245 MHz
        Memory                            : 877 MHz
    Default Applications Clocks
        Graphics                          : 1245 MHz
        Memory                            : 877 MHz
    Deferred Clocks
        Memory                            : N/A
    Max Clocks
        Graphics                          : 1380 MHz
        SM                                : 1380 MHz
        Memory                            :

In [7]:
!lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          80
On-line CPU(s) list:             0-79
Thread(s) per core:              2
Core(s) per socket:              20
Socket(s):                       2
NUMA node(s):                    2
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           85
Model name:                      Intel(R) Xeon(R) Gold 6148 CPU @ 2.40GHz
Stepping:                        4
CPU MHz:                         1000.066
CPU max MHz:                     3700.0000
CPU min MHz:                     1000.0000
BogoMIPS:                        4800.00
Virtualization:                  VT-x
L1d cache:                       1.3 MiB
L1i cache:                       1.3 MiB
L2 cache:                        40 MiB
L3 cache:                   